<a href="https://colab.research.google.com/github/jgbrenner/psychometrics/blob/main/psychometricsC9_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary Python packages
!pip install openai rpy2

In [3]:
# Import necessary Python libraries
import os
import openai
import pandas as pd
import numpy as np
import json
import requests
import re
from google.colab import drive, userdata

In [4]:
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

Pandas version: 2.2.2
NumPy version: 1.26.4


In [5]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the path where the R library will be saved in Google Drive
library_path = '/content/drive/MyDrive/R_libraries'

# Create the directory if it doesn't exist
if not os.path.exists(library_path):
    os.makedirs(library_path)

Mounted at /content/drive


In [6]:
import rpy2
print(f"rpy2 version: {rpy2.__version__}")

rpy2 version: 3.4.2


In [7]:
%load_ext rpy2.ipython

In [8]:
%%R

# Set the custom library path
library_path <- '/content/drive/MyDrive/R_libraries'
.libPaths(library_path)

# Verify that the custom library path is set correctly
print("Current R Library Paths:")
print(.libPaths())

[1] "Current R Library Paths:"
[1] "/content/drive/MyDrive/R_libraries" "/usr/local/lib/R/site-library"     
[3] "/usr/lib/R/site-library"            "/usr/lib/R/library"                


In [9]:
%%R

install_if_missing <- function(package, lib_path) {
  # Ensure the library path exists
  if (!dir.exists(lib_path)) {
    dir.create(lib_path, recursive = TRUE)
    message(paste("Created library path at:", lib_path))
  }

  # Check if the package is available
  if (!requireNamespace(package, quietly = TRUE)) {
    message(paste("Package", package, "not found. Installing to custom library path..."))

    # Attempt to install the package
    tryCatch({
      install.packages(package, lib = lib_path, repos = "https://cran.rstudio.com/", dependencies = TRUE)
    }, error = function(e) {
      stop(paste("Failed to install package", package, ":", e$message))
    })
  }

  # Load the package from the custom library path
  tryCatch({
    library(package, lib.loc = lib_path, character.only = TRUE)
    message(paste("Package", package, "successfully installed and loaded."))
  }, error = function(e) {
    stop(paste("Failed to load package", package, ":", e$message))
  })
}


In [10]:

%%R

# Verify installed packages
message("Installed packages in custom library path:")
installed_pkgs <- installed.packages(lib.loc = library_path)
print(installed_pkgs[, "Package"])

# Verify library paths
message("Current R library paths:")
print(.libPaths())


           abind          aricode              arm            bench 
         "abind"        "aricode"            "arm"          "bench" 
              BH              car          carData        checkmate 
            "BH"            "car"        "carData"      "checkmate" 
            clue             coda          corpcor         corrplot 
          "clue"           "coda"        "corpcor"       "corrplot" 
            covr          cowplot             CVXR       dendextend 
          "covr"        "cowplot"           "CVXR"     "dendextend" 
         DEoptim            Deriv             doBy            dplyr 
       "DEoptim"          "Deriv"           "doBy"          "dplyr" 
       ECOSolveR           EGAnet          fdrtool          foreach 
     "ECOSolveR"         "EGAnet"        "fdrtool"        "foreach" 
         Formula         fungible           future     future.apply 
       "Formula"       "fungible"         "future"   "future.apply" 
              GA           GGally 

[1] "/content/drive/MyDrive/R_libraries" "/usr/local/lib/R/site-library"     
[3] "/usr/lib/R/site-library"            "/usr/lib/R/library"                


In [26]:
r(f'''
install.packages("dplyr", lib = "{library_path}", repos = "http://cran.us.r-project.org", dependencies = TRUE)
library(dplyr, lib.loc = "{library_path}")
''')





































































	‘/tmp/RtmpoXhZjl/downloaded_packages’

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




'dplyr','reticula...,'tools',...,'datasets','methods','base'


In [27]:
%%R

library("dplyr", lib.loc = "/content/drive/MyDrive/R_libraries")

In [28]:
%%R
packageVersion("dplyr")

[1] ‘1.1.4’


In [11]:
# Verify rpy2 version ( 3.4.2 works )
import rpy2
print(f"rpy2 version: {rpy2.__version__}")

rpy2 version: 3.4.2


In [12]:
# Fetching the OpenAI API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    raise ValueError("OPENAI_API_KEY is not set or invalid.")

# Import OpenAI Client
from openai import Client
client = Client()

In [13]:
# 1. **Generate Scale Items Using OpenAI API**
messages = [
    {
        "role": "system",
        "content": "Jesteś ekspertem w psychometrii, który tworzy pytania testowe."
    },
    {
        "role": "user",
        "content": (
            "Wygeneruj po 3 stwierdzenia dla każdego z następujących konstruktów: "
            "Perfekcjonizm skierowany na siebie: Tendencja do wymagania doskonałości od siebie samego, "
            "Perfekcjonizm skierowany na innych: Stawianie wysokich oczekiwań wobec innych i krytyczna ocena ich osiągnięć, "
            "Perfekcjonizm społecznie narzucony: Przekonanie, że inni oczekują od nas doskonałości. "
            "Niech stwierdzenia będą zwięzłe i jasne,  odpowiednie do oceny w skali Likerta. "
            "Provide the output ONLY in JSON format as a list of dictionaries, "
            "without any additional text or explanation. "
            "Each dictionary should have keys 'construct' and 'item'."
        )
    }
]

try:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=8048,
        top_p=1,
        stream=False
    )
    response_content = completion.choices[0].message.content
    print("\nGenerated Items:\n", response_content)
except Exception as e:
    print(f"An error occurred: {e}")
    response_content = ""


Generated Items:
 ```json
[
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Czuję, że zawsze muszę osiągać doskonałość w tym, co robię."
    },
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Często krytycznie oceniam swoje osiągnięcia, nawet jeśli są wystarczające."
    },
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Nie akceptuję swoich błędów i porażek."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Oczekuję, że inni będą osiągać najwyższe standardy w swojej pracy."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Często krytykuję innych za ich niedoskonałości."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Uważam, że ludzie w moim otoczeniu powinni być idealni."
    },
    {
        "construct": "Perfekcjonizm społecznie narzucony",
        "item": "Czuj

In [14]:
# 2. **Parse and Validate Data with Fallback**
try:
    # Attempt to parse the response content directly
    generated_items = json.loads(response_content)
    items_df = pd.DataFrame(generated_items)
except json.JSONDecodeError as e:
    print(f"JSON parsing failed: {e}")
    # Attempt to extract JSON-like content using regex
    json_match = re.search(r'\[.*\]', response_content, re.DOTALL)  # Match JSON array
    if json_match:
        json_str = json_match.group(0)  # Extract the matched JSON-like content
        print("Extracted JSON String:")
        print(json_str)
        try:
            # Attempt to parse the extracted string as JSON
            generated_items = json.loads(json_str)
            items_df = pd.DataFrame(generated_items)
        except json.JSONDecodeError as e2:
            print(f"Second JSON decoding attempt failed: {e2}")
            items_df = None
    else:
        print("No valid JSON found in the response.")
        items_df = None

# Check if the DataFrame was successfully created
if items_df is not None:
    print("\nInitial Items DataFrame:")
    print(items_df.head())
else:
    raise ValueError("Failed to parse JSON. Ensure the LLM response is valid.")


JSON parsing failed: Expecting value: line 1 column 1 (char 0)
Extracted JSON String:
[
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Czuję, że zawsze muszę osiągać doskonałość w tym, co robię."
    },
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Często krytycznie oceniam swoje osiągnięcia, nawet jeśli są wystarczające."
    },
    {
        "construct": "Perfekcjonizm skierowany na siebie",
        "item": "Nie akceptuję swoich błędów i porażek."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Oczekuję, że inni będą osiągać najwyższe standardy w swojej pracy."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Często krytykuję innych za ich niedoskonałości."
    },
    {
        "construct": "Perfekcjonizm skierowany na innych",
        "item": "Uważam, że ludzie w moim otoczeniu powinni być idealni."
    },
    {
        "construct": 

In [15]:
# Remove duplicates and empty rows
items_df.drop_duplicates(inplace=True)
items_df.dropna(inplace=True)

# Map constructs to abbreviations
construct_abbreviations = {
    "Perfekcjonizm skierowany na siebie": "PSS",
    "Perfekcjonizm skierowany na innych": "PSI",
    "Perfekcjonizm społecznie narzucony": "PSP"
}

# Create descriptive unique labels for items
items_df['item_label'] = items_df.apply(
    lambda row: f"{construct_abbreviations[row['construct']]}_{row.name+1}", axis=1
)

# Inspect the cleaned DataFrame with improved labels
print("\nCleaned Items DataFrame with Improved Labels:")
print(items_df.head())



Cleaned Items DataFrame with Improved Labels:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item item_label  
0  Czuję, że zawsze muszę osiągać doskonałość w t...      PSS_1  
1  Często krytycznie oceniam swoje osiągnięcia, n...      PSS_2  
2             Nie akceptuję swoich błędów i porażek.      PSS_3  
3  Oczekuję, że inni będą osiągać najwyższe stand...      PSI_4  
4    Często krytykuję innych za ich niedoskonałości.      PSI_5  


In [16]:
# Extract item texts for embedding
item_texts = items_df['item'].tolist()

# Set up the API endpoint and headers for OpenAI embeddings
embedding_endpoint = "https://api.openai.com/v1/embeddings"
embedding_model = "text-embedding-3-small"

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json"
}

# Prepare the data payload
data = {
    "model": embedding_model,
    "input": item_texts
}

try:
    response = requests.post(embedding_endpoint, headers=headers, json=data)
    if response.status_code == 200:
        response_data = response.json()
        embeddings = [item['embedding'] for item in response_data['data']]
        embeddings_array = np.array(embeddings)
        print(f"Embeddings generated successfully. Shape: {embeddings_array.shape}")

        # Save embeddings for future use
        np.save("embeddings.npy", embeddings_array)
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        embeddings_array = None
except Exception as e:
    print(f"An error occurred during embedding generation: {e}")
    embeddings_array = None


Embeddings generated successfully. Shape: (9, 1536)


In [17]:
# Check alignment between items and embeddings
if embeddings_array is not None and embeddings_array.shape[0] != len(items_df):
    print(f"Embeddings count ({embeddings_array.shape[0]}) does not match items count ({len(items_df)}). Adjusting items_df.")
    items_df = items_df.iloc[:embeddings_array.shape[0]]
    items_df.reset_index(drop=True, inplace=True)

# Add embeddings back to DataFrame for inspection
items_df['embedding'] = list(embeddings_array)
print("\nItems DataFrame with Embeddings:")
print(items_df.head())



Items DataFrame with Embeddings:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item item_label  \
0  Czuję, że zawsze muszę osiągać doskonałość w t...      PSS_1   
1  Często krytycznie oceniam swoje osiągnięcia, n...      PSS_2   
2             Nie akceptuję swoich błędów i porażek.      PSS_3   
3  Oczekuję, że inni będą osiągać najwyższe stand...      PSI_4   
4    Często krytykuję innych za ich niedoskonałości.      PSI_5   

                                           embedding  
0  [0.050815463, 0.04002952, -0.030819692, 0.0326...  
1  [0.03050622, 0.027505938, -0.029036283, 0.0192...  
2  [0.03416719, 0.03534347, -0.012865537, 0.03390...  
3  [0.024419991, 0.01680323, -0.014380193, 0.0381...  
4  [0.015706629, 0.032804854, -0.01651498

In [18]:

# Save DataFrame as CSV for inspection in R
items_df.to_csv("psychometric_items_with_embeddings.csv", index=False)

# Save embeddings as a CSV file
np.savetxt("embeddings.csv", embeddings_array, delimiter=",")
print("Embeddings saved as CSV.")

# Inspect saved file locations and sizes
print("\nSaved files:")
print("Items DataFrame (CSV):", os.path.abspath("psychometric_items_with_embeddings.csv"))
print("Embeddings (CSV):", os.path.abspath("embeddings.csv"))


Embeddings saved as CSV.

Saved files:
Items DataFrame (CSV): /content/psychometric_items_with_embeddings.csv
Embeddings (CSV): /content/embeddings.csv


In [19]:

# Reload and inspect the CSV
reloaded_items_df = pd.read_csv("psychometric_items_with_embeddings.csv")
print("\nReloaded Items DataFrame:")
print(reloaded_items_df.head())
print("\nColumns:", reloaded_items_df.columns)



Reloaded Items DataFrame:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item item_label  \
0  Czuję, że zawsze muszę osiągać doskonałość w t...      PSS_1   
1  Często krytycznie oceniam swoje osiągnięcia, n...      PSS_2   
2             Nie akceptuję swoich błędów i porażek.      PSS_3   
3  Oczekuję, że inni będą osiągać najwyższe stand...      PSI_4   
4    Często krytykuję innych za ich niedoskonałości.      PSI_5   

                                           embedding  
0  [ 0.05081546  0.04002952 -0.03081969 ... -0.03...  
1  [ 0.03050622  0.02750594 -0.02903628 ... -0.04...  
2  [ 0.03416719  0.03534347 -0.01286554 ... -0.00...  
3  [ 0.02441999  0.01680323 -0.01438019 ... -0.00...  
4  [ 0.01570663  0.03280485 -0.01651498 ... -0.0

In [20]:

# Reload and inspect embeddings
reloaded_embeddings = np.load("embeddings.npy")
print("\nReloaded Embeddings Shape:", reloaded_embeddings.shape)
print("\nFirst Row of Reloaded Embeddings:", reloaded_embeddings[0])



Reloaded Embeddings Shape: (9, 1536)

First Row of Reloaded Embeddings: [ 0.05081546  0.04002952 -0.03081969 ... -0.03051586 -0.02069838
 -0.03402889]


In [21]:
%%R

# Load psychometric items and embeddings
library(reticulate)

# Load the CSV for items
items_df <- read.csv("psychometric_items_with_embeddings.csv")

# Load the CSV file for embeddings
embeddings_array <- as.matrix(read.csv("embeddings.csv", header = FALSE))

# Inspect loaded data
print("Loaded Items DataFrame:")
print(head(items_df))

print("Loaded Embeddings Array Dimensions:")
print(dim(embeddings_array))


[1] "Loaded Items DataFrame:"
                           construct
1 Perfekcjonizm skierowany na siebie
2 Perfekcjonizm skierowany na siebie
3 Perfekcjonizm skierowany na siebie
4 Perfekcjonizm skierowany na innych
5 Perfekcjonizm skierowany na innych
6 Perfekcjonizm skierowany na innych
                                                                        item
1                Czuję, że zawsze muszę osiągać doskonałość w tym, co robię.
2 Często krytycznie oceniam swoje osiągnięcia, nawet jeśli są wystarczające.
3                                     Nie akceptuję swoich błędów i porażek.
4         Oczekuję, że inni będą osiągać najwyższe standardy w swojej pracy.
5                            Często krytykuję innych za ich niedoskonałości.
6                    Uważam, że ludzie w moim otoczeniu powinni być idealni.
  item_label
1      PSS_1
2      PSS_2
3      PSS_3
4      PSI_4
5      PSI_5
6      PSI_6
                                                                        embedding

In [22]:
%%R

# Compute the correlation matrix from embeddings
cor_matrix <- cor(t(embeddings_array))
cor_matrix[is.na(cor_matrix)] <- 0  # Replace NA values with 0

# Inspect the correlation matrix
print("Correlation Matrix Dimensions:")
print(dim(cor_matrix))

print("Sample Correlation Matrix:")
print(cor_matrix[1:5, 1:5])


[1] "Correlation Matrix Dimensions:"
[1] 9 9
[1] "Sample Correlation Matrix:"
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 1.0000000 0.5938255 0.4148820 0.5772500 0.4803866
[2,] 0.5938255 1.0000000 0.4802275 0.4277503 0.6976542
[3,] 0.4148820 0.4802275 1.0000000 0.3082386 0.5493691
[4,] 0.5772500 0.4277503 0.3082386 1.0000000 0.4190762
[5,] 0.4803866 0.6976542 0.5493691 0.4190762 1.0000000


In [23]:
%%R

# Apply Unique Variable Analysis (UVA) to identify redundant items
library(EGAnet)

uva_result <- UVA(
    data = cor_matrix,
    n = 9,
    method = "wTO",  # Weighted Topological Overlap
    threshold = 0.20  # Adjust threshold as needed
)

# Print redundant items
redundant_items <- uva_result$redundant
print("Redundant Items Identified:")
print(redundant_items)

# Remove redundant items from the embeddings and correlation matrix
embeddings_matrix <- embeddings_array[!rownames(embeddings_array) %in% redundant_items, ]
cor_matrix <- cor_matrix[!rownames(cor_matrix) %in% redundant_items, !colnames(cor_matrix) %in% redundant_items]


EGAnet (version 2.1.0) 

For help getting started, see <https://r-ega.net> 

For bugs and errors, submit an issue to <https://github.com/hfgolino/EGAnet/issues>



[1] "Redundant Items Identified:"
NULL


In [24]:
%%R

# Perform EGA to detect latent constructs
ega_result <- EGA(
    data = cor_matrix,
    model = "glasso",    # Graphical Lasso
    algorithm = "walktrap"  # Community detection algorithm
)

# Inspect EGA results
print("EGA Communities Detected:")
print(ega_result$wc)  # Community assignments


  length of 'dimnames' [2] not equal to array extent








Error in dimnames(data) <- `*vtmp*` : 
  length of 'dimnames' [2] not equal to array extent


In [25]:
%%R

# Check dimensions of the correlation matrix
print("Correlation Matrix Dimensions:")
print(dim(cor_matrix))

# Check row and column names of the correlation matrix
print("Correlation Matrix Row Names:")
print(rownames(cor_matrix))
print("Correlation Matrix Column Names:")
print(colnames(cor_matrix))

# Check item labels in items_df
print("Item Labels in Items DataFrame:")
print(items_df$item_label)

# Align correlation matrix dimnames with item labels
rownames(cor_matrix) <- items_df$item_label
colnames(cor_matrix) <- items_df$item_label


[1] "Correlation Matrix Dimensions:"
[1] 0 0
[1] "Correlation Matrix Row Names:"
NULL
[1] "Correlation Matrix Column Names:"
NULL
[1] "Item Labels in Items DataFrame:"
[1] "PSS_1" "PSS_2" "PSS_3" "PSI_4" "PSI_5" "PSI_6" "PSP_7" "PSP_8" "PSP_9"


  length of 'dimnames' [1] not equal to array extent




Error in dimnames(x) <- dn : 
  length of 'dimnames' [1] not equal to array extent


In [26]:
%%R

# Check dimensions of the embeddings array
print("Embeddings Array Dimensions:")
print(dim(embeddings_array))

# Inspect the first few rows of the embeddings array
print("First Few Rows of Embeddings Array:")
print(embeddings_array[1:5, 1:5])


[1] "Embeddings Array Dimensions:"
[1]    9 1536
[1] "First Few Rows of Embeddings Array:"
             V1         V2          V3         V4           V5
[1,] 0.05081546 0.04002952 -0.03081969 0.03268065  0.013387482
[2,] 0.03050622 0.02750594 -0.02903628 0.01921993  0.024203615
[3,] 0.03416719 0.03534347 -0.01286554 0.03390988  0.004567265
[4,] 0.02441999 0.01680323 -0.01438019 0.03813649  0.018678450
[5,] 0.01570663 0.03280485 -0.01651498 0.03446249 -0.006697061


In [27]:
%%R

# Recompute the correlation matrix
cor_matrix <- cor(t(embeddings_array))  # Compute correlations between items
cor_matrix[is.na(cor_matrix)] <- 0      # Replace any NA values with 0

# Inspect the recomputed correlation matrix
print("Correlation Matrix Dimensions:")
print(dim(cor_matrix))

print("Sample of Correlation Matrix:")
print(cor_matrix[1:5, 1:5])


[1] "Correlation Matrix Dimensions:"
[1] 9 9
[1] "Sample of Correlation Matrix:"
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 1.0000000 0.5938255 0.4148820 0.5772500 0.4803866
[2,] 0.5938255 1.0000000 0.4802275 0.4277503 0.6976542
[3,] 0.4148820 0.4802275 1.0000000 0.3082386 0.5493691
[4,] 0.5772500 0.4277503 0.3082386 1.0000000 0.4190762
[5,] 0.4803866 0.6976542 0.5493691 0.4190762 1.0000000


In [28]:
%%R


# Assign item labels as row and column names
rownames(cor_matrix) <- items_df$item_label
colnames(cor_matrix) <- items_df$item_label

# Verify alignment
print("Aligned Correlation Matrix Dimensions:")
print(dim(cor_matrix))

print("Correlation Matrix Row Names:")
print(rownames(cor_matrix)[1:5])

print("Correlation Matrix Column Names:")
print(colnames(cor_matrix)[1:5])


[1] "Aligned Correlation Matrix Dimensions:"
[1] 9 9
[1] "Correlation Matrix Row Names:"
[1] "PSS_1" "PSS_2" "PSS_3" "PSI_4" "PSI_5"
[1] "Correlation Matrix Column Names:"
[1] "PSS_1" "PSS_2" "PSS_3" "PSI_4" "PSI_5"


In [29]:
%%R


# Perform EGA
ega_result <- EGA(
    data = cor_matrix,
    model = "glasso",    # Graphical Lasso
    algorithm = "walktrap",  # Community detection algorithm
    n = 9                # Sample size (number of rows in embeddings_array)
)

# Inspect EGA results
print("EGA Communities Detected:")
print(ega_result$wc)  # Community memberships for each item


[1] "EGA Communities Detected:"
Algorithm:  Louvain

Number of communities:  1

PSS_1 PSS_2 PSS_3 PSI_4 PSI_5 PSI_6 PSP_7 PSP_8 PSP_9 
    1     1     1     1     1     1     1     1     1 
